Penerapan Algoritma Genetik untuk menyelesaikan TSP

memanggil library

In [0]:
import numpy as np 
import random
from array import *

Membuat daftar kota 

In [0]:
daftarKota = [[0,0], [5,6], [1, 4], [3,4],[7,7]]
for i in daftarKota:
    for j in i:
        print(j,end = " ")
    print()

jumlahKota =  len(daftarKota)   
print(jumlahKota)

0 0 
5 6 
1 4 
3 4 
7 7 
5


Inisialisasi Populasi secara acak

Array P diinisialisasi sebagai array integer

Solusi awal dibuat dengan random permutation

In [0]:
def initPopulasi(ukuranPopulasi,jumlahKota):
  P = np.empty((ukuranPopulasi,jumlahKota))
  P = P.astype('int')
  for i in range(ukuranPopulasi):
    I = np.random.permutation(jumlahKota)
    for j in range(jumlahKota):
      P[i][j]=I[j]
  return P


ukuranPopulasi = 4
P = initPopulasi (ukuranPopulasi,jumlahKota)
print(P)



[[2 3 4 0 1]
 [4 3 2 1 0]
 [2 3 0 4 1]
 [2 3 4 1 0]]


Membuat fungsi objektif

Jarak yang diukur adalah jarak euclidian

In [0]:
def fungsiObjectif(populasi,daftarKota):
  sz = populasi.shape
  ukuranPopulasi = sz[0]
  jumlahKota = sz[1]

  matrikJarak = np.empty((ukuranPopulasi))
  matrikJarak  = matrikJarak.astype('float')
  n = len(daftarKota)

  for i in range(ukuranPopulasi):
    jarak = 0
    for j in range(n-1):
      jrkX = daftarKota[populasi[i][j]][0]-daftarKota[populasi[i][j+1]][0]
      jrkY = daftarKota[populasi[i][j]][1]-daftarKota[populasi[i][j+1]][1]
      d = np.sqrt(jrkX**2 + jrkY**2 )   
      jarak = jarak  + d
      
    dn =  np.sqrt((daftarKota[populasi[i][n-1]][0]-daftarKota[populasi[i][0]][0])**2+ (daftarKota[populasi[i][n-1]][1]-daftarKota[populasi[i][0]][1])**2 )
    jarak = jarak  + dn
    matrikJarak[i] = jarak

  return matrikJarak

fitP = fungsiObjectif(P,daftarKota)
print(fitP)
#print(fitP.dtype)



[29.18188057 29.18188057 23.60769887 21.16942328]


Seleksi menggunakan metode tournamen 

Ukuran turnamen k = 2

Jumlah induk yang dipilih adalah 2

In [0]:

def seleksi (P,fsObj):
  jmlInduk = 2
  sz = P.shape
  ukuranPopulasi = sz[0]
  jumlahKota = sz[1]

  Induk = np.empty((jmlInduk,jumlahKota))
  Induk = Induk.astype('int')
  
  for i in range (2):
    # memilih dua individu secara acak 
    r1 = random.randint(0,ukuranPopulasi-1)
    r2 = random.randint(0,ukuranPopulasi-1)
    
    if  fsObj[r1] < fsObj [r2]:
      for j in range(jumlahKota):
        Induk [i][j] = P[r1][j]
    else:
      for j in range(jumlahKota):
        Induk [i][j] = P[r2][j]
  return Induk

Induk = seleksi(P,fitP)
print(Induk)

[[2 3 0 4 1]
 [2 3 4 1 0]]


Rekombinasi menggunaka metode Partially Mixed Crossover (PMX)
Jumlah anak yang dihasilkan adalah 2

In [0]:
#crossover 

def OrderXO(Induk):
  sz = Induk.shape
  brs = sz[0]
  kol = sz[1]
  #print(sz, brs, kol)

  Anak = np.empty((2,kol))
  Anak = Anak.astype('int')

  r1 = random.randint(1,kol-2)

  for i0 in range(r1): 

    Anak [0][i0] = Induk[0][i0]
    Anak [1][i0] = Induk[1][i0]

  batasCek1 = r1
  batasCek2 = r1

  for i in range(kol):
    m = (i+r1)%kol

    n1 = Induk[0][i] 
    n2 = Induk[1][i]

    #mengecek apakah kota tertentu sudah ada di anak
    idxAda1 = 0
    idxAda2 = 0
    for j1 in range(batasCek1): 
      
      if batasCek1>(kol-1):
        idxAda1 = 1

      if Anak [0][j1] ==n2: 
        idxAda1 = 1
            
    for j2 in range(batasCek2): 
      
      if batasCek2>(kol-1):
        idxAda2 = 1

      if Anak [1][i0]  == n1: 
        idxAda2 = 1

    if idxAda1 == 0:
      Anak[0][batasCek1] = n2
      batasCek1 = batasCek1 +1
   
    if idxAda2 == 0:
      Anak[1][batasCek2] = n1
      batasCek2 = batasCek2 +1
   
  return Anak

AnakCX = PMX(Induk)
print("Anak :",AnakCX)


Anak : [[2 3 4 1 0]
 [2 3 0 4 1]]


Mutasi dengan Swap

index kota yang di swap pada kedua anak sama

In [0]:
def MutasiSwap(Induk):
  sz = Induk.shape
  brs = sz[0]
  kol = sz[1]

  Anak = np.empty((2,kol))
  Anak = Anak.astype('int')

  Anak = Induk

  r1 = random.randint(0,kol-1)
  r2 = random.randint(0,kol-1)

  while r1 == r2:
    r2 = random.randint(0,kol-1)

  I1rs1 = Induk [0][r1]
  I1rs2 = Induk [0][r2]
  I2rs1 = Induk [1][r1]
  I2rs2 = Induk [1][r2] 

  Anak [0][r1] = I1rs2
  Anak [0][r2] = I1rs1
  Anak [1][r1] = I2rs2  
  Anak [1][r2] = I2rs1

  return Anak

print("AnakCX :",AnakCX)
AnakM = MutasiSwap(AnakCX)
print("AnakM :",AnakM)


AnakCX : [[3 4 0 2 1]
 [3 0 1 2 4]]
AnakM : [[3 0 4 2 1]
 [3 1 0 2 4]]


Elitism (steady state replacement)

In [0]:
def Elitism(P,Anak,fitP,fitAnak):
  szP = P.shape
  brsP = szP[0]
  kolP = szP[1]
  szA = Anak.shape
  brsA = szA[0]
  kolA = szA[1]

  #print (fitP)
  for i in range(brsA):
    iJelek = fitP.max()
    idxJelek = fitP.argmax()

    if fitAnak[i]<iJelek:
      fitP[idxJelek] = fitAnak[i]
      for j in range (kolP):
        P[idxJelek][j] = Anak[i][j]

  return [P,fitP]

fitAnak= fungsiObjectif(AnakM,daftarKota)
print(P)
print(AnakM)
print(fitP)
print(fitAnak)
P = Elitism (P,AnakM,fitP,fitAnak)
print(P)
print(fitP)


[[2 3 4 0 1]
 [4 3 2 1 0]
 [2 3 0 4 1]
 [2 3 4 1 0]]
[[3 4 1 0 2]
 [3 0 4 1 2]]
[29.18188057 29.18188057 23.60769887 21.16942328]
[21.16942328 23.60769887]
[array([[3, 4, 1, 0, 2],
       [3, 0, 4, 1, 2],
       [2, 3, 0, 4, 1],
       [2, 3, 4, 1, 0]]), array([21.16942328, 23.60769887, 23.60769887, 21.16942328])]
[21.16942328 23.60769887 23.60769887 21.16942328]


Prosedur GA 

In [0]:
# Inisialisasi paramater
PCX = 0.95
Pm = 0.01
ukuranPopulasi = 4
max_generasi = 5

# Inisialisasi populasi 

P = initPopulasi (ukuranPopulasi,jumlahKota)
print("P0 :",P)

#melakukan evaluasi P
fitP = fungsiObjectif(P,daftarKota)
print("fitness populasi awal:",fitP)

print("fitness terbaik awal:",fitP.min())
print("rute awal :",P[fitP.argmin()])


for i in range(max_generasi):
  print("generasi :",i)
  Induk = seleksi(P,fitP)
  #print("Induk :",Induk)

  Anak = Induk

  rxo = random.random()

  if rxo < PCX:
    Anak = OrderXO(Induk)
    #print("AnakXO :",Anak)

    rm = random.random()
    if rm < Pm:
      Anak = MutasiSwap(Anak)
      #print("AnakR :",Anak)

  fitAnak = fungsiObjectif(Anak,daftarKota)
  #print("fitAnak :",fitAnak)

  print("fitness:",fitAnak.min())

  [P, fitP]= Elitism (P,Anak,fitP,fitAnak)   
  
print("fitness populasi akhir:",fitP)
print("fitness akhir:",fitP.min())
print("rute akhir :",P[fitP.argmin()])




P0 : [[2 3 0 4 1]
 [2 4 1 0 3]
 [4 3 1 2 0]
 [4 3 0 2 1]]
fitness populasi awal: [23.60769887 23.75452159 26.32316364 20.83130956]
fitness terbaik awal: 20.83130955811703
rute awal : [4 3 0 2 1]
generasi : 0
fitness: 21.02260056222933
generasi : 1
fitness: 23.754521585905813
generasi : 2
fitness: 21.169423279024105
generasi : 3
fitness: 23.607698869111033
generasi : 4
fitness: 24.5395134906164
fitness populasi akhir: [21.16942328 21.02260056 21.16942328 20.83130956]
fitness akhir: 20.83130955811703
rute akhir : [4 3 0 2 1]
